# 6. Flask - API
When we need our systems to communicate directly with other machines (bots, drones, probes...), we must create a special connection point - an "ENDPOINT". We will therefore build an Application Programming Interface (API), which will serve as a standardized data channel for our machine-to-machine communication. This will allow external systems to securely read and manipulate our data without needing access to our graphical interface.

- Creating an `API` (Application Programming Interface) & `RESTful API` (Representational State Transfer) principles
- `HTTP methods` GET, POST, **PUT**, **DELETE** for API endpoints

`Application structure`:
- my_project/
  - my_app.py
  - **data/**
    - **test_data.py**
  - templates/
  - static/

## 6.1. RESTful API Principles
For communication between different systems to work reliably, everyone must follow the same rules and protocols. For APIs, the REST architecture is such a standard, defining a set of best practices for designing clear and predictable data interfaces. Let's look at the basic rules that will ensure our API is understandable to any system that wants to connect to it.

### REST RULES:

- `Stateless` - every client request to the server must contain ALL the information needed for successful processing = it does not rely on previous conversation.
- `Client-server` - the client (= the one who asks and sends data requests) and the server (= the one who responds and sends responses based on the request) are independent = the client handles the user interface and data display, the server supplies and processes data based on requests.
- `Cacheable` - the response object must state (cacheable/non-cacheable) whether, and for how long, the data is "valid".
- `Layered system` - there can be multiple intermediate layers between the client and the server (other servers, satellites ..) for better load balancing = the client does not know if it is communicating directly with the end server.

- `Uniform interface`- a unified interface and principles for client-server communication:
  - **Identification** - clear and unambiguous identification of information and data resources = unique URI (Uniform Resource Identifier) addresses - usually unique URL addresses.
    `"/sensors/temperature"` is the resource for data from the temperature sensor.
  - **Representation** - we never work directly with the resource, but with its representation = usually a response in JSON (JavaScript Object Notation) format.
  - **Self-descriptive messages** - each request and each response contain the information needed to understand them.
    - = the request uses HTTP methods to manipulate resources/data: GET (to retrieve), POST (to create), PUT (to update), and DELETE (to delete).
    - = the response in the response header: `Content-Type: application/json` (the data being sent is in JSON format).

### Summary:
- Data is available at selected URLs in the form of JSON files.
- Data can be obtained as a response when we ask correctly using an HTTP method - e.g., PUT to update data.
- These are just principles of polite communication :)

## 6.2. JSON Response for API Endpoints
For our data transmissions to be understandable by any system, they must be structured in a standardized format. We will explore how to use the `jsonify()` function to package our data into a universal JSON container and how to use `make_response()` to add special metadata and instructions to the data shipment.

- `jsonify()`:
  - Creates a response in `JSON` format.
  - We pass ARG/ARGs or KWARG/KWARGs, and the resulting list/dict will be converted to JSON.
  - We can add **status codes** (200, 300, 400, etc.).
  - It `automatically adds` to the headers: **`"Content-Type: application/json"`**.
  - Primarily for the rapid creation of API endpoints.

- `make_response()`
  - Gives us `more options for setting up the response` (it can be text, html, json ..).
  - Allows specifying **headers**, **status codes**, precisely defining the **response body**, and setting **cookies**.

## 6.3. HTTP Status Codes
Every communication between Mission Control and our systems in the field must include a clear `confirmation of status`. We will learn to use standardized `HTTP status codes`, which serve as quick and unambiguous signals about whether the mission (request) was successful, failed, or requires further action.

#### **Server responses** to a **client request**:

- `100` series = we are preparing the response, please wait.
- `200` series = everything is OK, you should have the data.
- `300` series = multiple possible responses.
  - 301 = redirection, etc.

- `400` series (Client Error - error on the user's side):
  - 400 = bad request.
  - 401 = unauthorized request.
  - 403 = forbidden.
  - 404 = not found, etc.

- `500` series (Server Error - error on the server's side):
  - 500 = internal server error.
  - 503 = service unavailable, etc.

## 6.4. Data for the API
Every `API endpoint` must return some data – without data, there's nothing to pass on. An API is not just an empty "gateway," but primarily an access point to information resources stored in the system. Data can come from various sources:
- `database` (e.g., SQLite, PostgreSQL …)
- `files` (e.g., .json, .csv, .py)
- `sensors`, remote systems, calculations, etc.

For testing and development purposes, we will use a simple `static data source` from which we will load information. We will then simply import and pass this data into our responses. In a real application, this data would be dynamic (e.g., from a database), but the principle remains the same – the API is the channel, not the source itself.

In [ ]:
# data/test_data.py

probe_data = [
  {
    "status": "ok",
    "vectors": {"x": 112, "y": -256, "z": 45},
    "velocity": 1250,
    "unit": "m/s"
  },
  {
    "status": "ok",
    "vectors": {"x": -480, "y": 301, "z": -110},
    "velocity": 980,
    "unit": "m/s"
  },
  {
    "status": "ok",
    "vectors": {"x": 205, "y": 150, "z": 215},
    "velocity": 1100,
    "unit": "m/s"
  },
  {
    "status": "ok",
    "vectors": {"x": 88, "y": -99, "z": -300},
    "velocity": 1420,
    "unit": "m/s"
  },
  {
    "status": "ok",
    "vectors": {"x": 0, "y": 510, "z": 15},
    "velocity": 895,
    "unit": "m/s"
  },
  {
    "status": "ok",
    "vectors": {"x": 315, "y": 96, "z": 30},
    "velocity": 1000,
    "unit": "m/s"
  },
  {
    "status": "ok",
    "vectors": {"x": -180, "y": -220, "z": 450},
    "velocity": 1310,
    "unit": "m/s"
  },
  {
    "status": "ok",
    "vectors": {"x": 25, "y": 75, "z": -50},
    "velocity": 950,
    "unit": "m/s"
  },
  {
    "status": "ok",
    "vectors": {"x": 333, "y": -123, "z": 280},
    "velocity": 1180,
    "unit": "m/s"
  },
  {
    "status": "ok",
    "vectors": {"x": -95, "y": 410, "z": -190},
    "velocity": 1055,
    "unit": "m/s"
  }
]

In [ ]:
# my_app.py

from flask import Flask, jsonify, make_response
from data.test_data import probe_data

app = Flask(__name__)

# -------------- CONFIGURATION --------------
app.config["DEBUG"] = True


# -------------- API - ENDPOINTS --------------
@app.route("/probe_velocities", methods=["GET"])
def probe_velocities():
    # Data passed to jsonify is obtained from a database, file, sensors ...
    probe_velocities = {"Velocities": [probe["velocity"] for probe in probe_data]}
    return jsonify(probe_velocities), 200


@app.route("/probe_info/<int:probe_id>", methods=["GET"]) # we can use a parameter in the URL
def probe_detail(probe_id):
    # Data passed to jsonify is obtained from a database, file, sensors ...
    if not 1 <= probe_id <= len(probe_data): # check if "probe_id" is within a valid range
        return jsonify({"error": "Probe ID is out of range. Not found."}), 404
    
    probe = probe_data[probe_id - 1]
    return jsonify(ID_Probe=probe_id, Details=probe), 200
        

@app.route('/probe_mothership', methods=['GET'])
def probe_mothership():
    # setting the BODY (response + status code using jsonify)
    response = make_response(jsonify(
        {"message": "No data to fetch, mothership empty, systems still running"}
        ), 202)
    
    # setting HEADERS
    response.headers["Cache-Control"] = "no-store" # cache setting
    response.headers["Content-Language"] = "en" # language setting

    # setting COOKIES
    response.set_cookie(
        key="session_mothership",
        value="50",
        max_age=3600,              # 1 hour
        httponly=True,
        secure=True,
        samesite="Lax"
    )
    return response
        

# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run() 

### **Practise I**

**API for Systems Diagnostics**
We will create a simple API for remote diagnostics of a planetary base's systems. We will prepare a `GET` endpoint that returns data and a `POST` endpoint that accepts a command and returns a custom response.

**Preparation:**
For this practise, use the following file as a base. Your task is to complete the code in the indicated places.
```python
from flask import Flask, jsonify, make_response, request
import uuid # generates a unique ID

app = Flask(__name__)

# --- Data Source ---
# In a real app, this would come from a database or live sensors.
SYSTEM_STATUSES = [
    {
        "system": "Life Support",
        "status": "Nominal",
        "details": "O2 Levels at 98%"
    },
    {
        "system": "Primary Power",
        "status": "Warning",
        "details": "Fluctuation detected in reactor core"
    },
    {
        "system": "Shield Generator",
        "status": "Active",
        "details": "Power draw at 75%"
    },
    {
        "system": "Communications Array",
        "status": "Offline",
        "details": "Signal lost with Orbiter One. Awaiting realignment."
    },
    {
        "system": "Drilling Laser",
        "status": "Standby",
        "details": "Awaiting command to initiate sequence."
    }
]

# --- API Endpoints ---

# Create the /api/status ENDPOINT here

# Create the /api/run_diagnostic ENDPOINT here


# --- App ---
if __name__ == "__main__":
    app.run(debug=True)
```

**Assignment:**

1.  **Create a `GET` endpoint for reading status:**
    * Build a new route `/api/status` that accepts `GET` requests.
    * It should return the `SYSTEM_STATUSES` dictionary in JSON format and add a status code.

2.  **Create a `POST` endpoint to run diagnostics:**
    * Build a new route `/api/run_diagnostic` that accepts `POST` requests.
    * It will simulate starting a long-running task.
    * Create a JSON response with a confirmation, e.g., `{"message": "Diagnostic sequence initiated"}`.
    * Use `make_response()` to "wrap" this JSON response so you can:
        * Set the status code to `202` (request has been accepted for processing).
        * Add a custom header `X-Request-ID`, whose value will be a unique ID (you can use `str(uuid.uuid4())`).

### **Practise II**
**Automated Diagnostic Script**

We will create an automated bot (an external Python script) that will connect to our running application's API, download the system status data, and automatically check for warnings. We will utilize the capabilities and data from our API.

**Preparation:**
1.  In one terminal, **keep the Flask application from the previous practise running**.
2.  Create a **new, separate file** `diagnostic_check.py`.
3.  Install the `requests` library.
    ```bash
    pip install requests
    ```

**Assignment:**
1.  In the `diagnostic_check.py` file, import the `requests` library.
2.  Define a variable with the URL of your local API endpoint: `http://127.0.0.1:5000/api/status`.
3.  Using `requests.get()`, send a `GET` request to this address.
4.  From the response object, get the data in JSON format (using the `.json()` method).
5.  Using a loop, iterate through the received data.
6.  If you encounter a `status` with the value `"Warning"` for any system, print a warning message to the console, for example:
    `WARNING: System 'Primary Power' reports a warning! Details: 'Fluctuation detected in reactor core'`

### **Project (Homework): Upgrade to an Automated Data Channel**

**Mission:** Our web-based information system is great for the crew, but autonomous systems and Mission Control on Earth need access to raw, real-time data, not formatted HTML pages. Our task is to create two new API endpoints that will serve as a standardized data channel for machine-to-machine communication.

`Basic application structure`:
- my_project/
  - my_app.py
  - routes.py
  - forms.py
  - database.py
  - models.py
  - **data/**
    - **asteroid.py**
    - **crew.py**
  - templates/
  - static/
  - config/

- We will keep our **SQLite DB** file in the default **instance** folder outside our application.

**1. Data Channel for the Information System:**
* Create a new API route `/api/crew_members` that will accept `GET` requests.
* Query the database to get **all** records of crew members.
* Convert them to a **list of dictionaries** and pass it to `jsonify()`.
* Return it as a JSON response with a `200 OK` status code.

**2. Telemetry of the Target Asteroid:**
* Create another API route `/api/target_asteroid` that will also accept `GET` requests.
* Return the data about `my_asteroid` as a JSON response using `jsonify()`.

**3. Data Stream Upgrade:**
* Move the data about the crew and the asteroid from the application file to separate files `asteroid.py` and `crew.py` in a new `data` folder - we will import them from there into our application.

**4. Transmission Verification:**
* After starting the application, verify the functionality of both new endpoints = enter them into a web browser. You should see clean, raw data in JSON format.

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom